# Initial setup

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# clone darknet
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have CUDA and OpenCV enabled
%cd darknet
!.\build.ps1 -UseVCPKG -EnableOPENCV -EnableCUDA -EnableCUDNN
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

# Configure darknet for custom training

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# modify values for using only one detection class
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!echo "Ear" > data/custom.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/custom.names\nbackup = /mydrive/sb_assignment2' > data/custom.data
!mkdir data/images

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

# Images preparation

In [ ]:
!unzip /mydrive/sb_assignment2/train_images.zip -d data/images

In [ ]:
import glob
images_list = glob.glob("data/images/*.png")
print(images_list)

In [ ]:
# Create train.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

# Training

In [ ]:
# Start training
!./darknet detector train data/custom.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show